![Renode](https://antmicro.com/OpenSource/assets/images/projects/renode.png)

## Intro + How to run this

This notebook shows how Renode, an open source IoT/embedded simulation framework from [Antmicro](https://antmicro.com) can be used for simulation a virtual Linux board with RISC-V and show some execution metrics.

This runs entirely in the browser - just use **Runtime -> Run all** and see the output!

For more backround, see the [README](https://github.com/mgielda/renode-notebooks-ipynb/tree/master) of the master repository.

In [ ]:
import os

!mkdir -p renode && cd renode && curl -L https://www.dropbox.com/s/b4eqhiqxgnn1vcx/renode.tar.gz | tar xz --strip 1
os.environ['PATH'] = os.getcwd()+"/renode:"+os.environ['PATH']

!pip3 install git+https://github.com/mgielda/pyrenode robotframework==3.1 netifaces requests psutil pyyaml matplotlib pandas numpy

## Housekeeping

Some initialization. Import a simplistic pyrenode lib to talk to Renode over telnet, then shutdown Renode just in case it was running and create a new instance.

In [ ]:
from pyrenode import *
shutdown_renode()
connect_renode() # this sets up a log file, and clears the simulation (just in case)

In [ ]:
tell_renode('i @scripts/single-node/hifive_unleashed.resc')
print(read_until('(hifive-unleashed)').strip())
tell_renode('machine EnableProfiler "demo"')
print(read_until('(hifive-unleashed)').strip())

# Put the UART output in the log - this allows us to wait for specific lines
tell_renode('showAnalyzer uart0 Antmicro.Renode.Analyzers.LoggingUartAnalyzer')
tell_renode('emulation RunFor "1"')
expect_log("SiFive GPIO chip registered 16 GPIOs", 60)

In [ ]:
shutdown_renode()
expect_log("Disposed.") # to be sure if buffer to dump file is flushed
!python3 renode/tools/metrics-analyzer/metrics-analyzer.py renode/demo

In [ ]:
from IPython.display import Image
Image('instructions.png')

In [ ]:
from IPython.display import Image
Image('memory.png')

In [ ]:
from IPython.display import Image
Image('exceptions.png')

In [ ]:
from IPython.display import Image
Image('peripherals.png')